# apriori analysis

### after clustering,it will generate k classes,each class will have a list of user_ids inside that class,next step is to apply basket anaysis on each classification,to get the rules of each class

In [1]:
import pandas as pd
import numpy as np

aisle = pd.read_csv('../../DataSets/aisles.csv')
department = pd.read_csv('../../DataSets/departments.csv')
order_products_prior = pd.read_csv('../../DataSets/order_products__prior.csv')
order_products_train = pd.read_csv('../../DataSets/order_products__train.csv')
orders = pd.read_csv('../../DataSets/orders.csv')
products = pd.read_csv('../../DataSets/products.csv')

# fresh fruits:24
# fresh vegetables:83
# packaged vegetables fruits:122

In [2]:
import time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
from mlxtend.frequent_patterns import association_rules

def Apriori(user_product_part1):
    
    user_product_dic={}
    for item in user_product_part1.itertuples():
        if(user_product_dic.get(item[2])):
            user_product_dic[item[2]].append(item[3])
        else:
            user_product_dic[item[2]] = [] 
            user_product_dic[item[2]].append(item[3])
    transaction = []
    for item in user_product_dic:
        transaction.append(user_product_dic[item])
#     print('transaction',transaction)
    te = TransactionEncoder()
    te_ary = te.fit(transaction).transform(transaction)
#     print('te_ary',te_ary)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    
    return df

In [3]:
def transform_associationRule_to_product(association_rule):
    
    relationship_rule = association_rule[['antecedents','consequents']]
    arr_relation=relationship_rule.values
    final_rules = []
    for item in arr_relation:
        pre_product_Arr = []
        after_product_Arr = []
        pre = item[0]
        after = item[1]
        for pre_item in pre:
            pre_product_Arr.append(products[products['product_id' ]== pre_item]['product_name'].values[0])
        for after_item in after:
            after_product_Arr.append(products[products['product_id' ]== after_item]['product_name'].values[0])
        tup = (pre_product_Arr,'----->',after_product_Arr)
        final_rules.append(tup)
    return final_rules

In [4]:
def transform_frequentItem_to_product(frequent_itemsets):
    arr_relation=frequent_itemsets.values
    final_arr = []
    for item in arr_relation:
        arr = []
        for it in item[1]:
            arr.append(products[products['product_id' ]== it]['product_name'].values[0])
        final_arr.append(arr)
    return final_arr

#  association items of all users (no any change)

In [5]:
order_user_id = orders[['order_id','user_id']]

order_products_id1 = order_products_train[['order_id','product_id']]
order_products_id2 = order_products_prior[['order_id','product_id']]
order_products_id = order_products_id1.append(order_products_id2, ignore_index=True)
# order_products_id = order_products_id2
order_products_id
user_product_id=order_user_id.merge(order_products_id,left_on='order_id', right_on='order_id')
user_product_id

,order_id,user_id,product_id
0,2539329,1,196
1,2539329,1,14084
2,2539329,1,12427
3,2539329,1,26088
4,2539329,1,26405
...,...,...,...
33819101,272231,206209,40603
33819102,272231,206209,15655
33819103,272231,206209,42606
33819104,272231,206209,37966


In [6]:
df = Apriori(user_product_id)
frequent_itemsets_all = apriori(df, min_support=0.1, use_colnames=True)
association_rule_all = association_rules(frequent_itemsets_all, metric="confidence", min_threshold=0.1)

In [7]:
transform_associationRule_to_product(association_rule_all)

[(['Bag of Organic Bananas'], '----->', ['Organic Strawberries']),
 (['Organic Strawberries'], '----->', ['Bag of Organic Bananas']),
 (['Bag of Organic Bananas'], '----->', ['Organic Baby Spinach']),
 (['Organic Baby Spinach'], '----->', ['Bag of Organic Bananas']),
 (['Bag of Organic Bananas'], '----->', ['Banana']),
 (['Banana'], '----->', ['Bag of Organic Bananas']),
 (['Bag of Organic Bananas'], '----->', ['Organic Hass Avocado']),
 (['Organic Hass Avocado'], '----->', ['Bag of Organic Bananas']),
 (['Banana'], '----->', ['Strawberries']),
 (['Strawberries'], '----->', ['Banana']),
 (['Organic Strawberries'], '----->', ['Organic Baby Spinach']),
 (['Organic Baby Spinach'], '----->', ['Organic Strawberries']),
 (['Organic Strawberries'], '----->', ['Banana']),
 (['Banana'], '----->', ['Organic Strawberries']),
 (['Limes'], '----->', ['Organic Strawberries']),
 (['Organic Strawberries'], '----->', ['Limes']),
 (['Organic Strawberries'], '----->', ['Organic Blueberries']),
 (['Organi

In [8]:
s = frequent_itemsets_all.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(s)


[['Banana'],
 ['Bag of Organic Bananas'],
 ['Organic Strawberries'],
 ['Organic Baby Spinach'],
 ['Large Lemon'],
 ['Limes'],
 ['Strawberries'],
 ['Organic Hass Avocado'],
 ['Organic Avocado'],
 ['Organic Blueberries'],
 ['Organic Garlic'],
 ['Organic Yellow Onion'],
 ['Organic Zucchini'],
 ['Organic Raspberries'],
 ['Bag of Organic Bananas', 'Organic Strawberries'],
 ['Yellow Onions'],
 ['Cucumber Kirby'],
 ['Organic Grape Tomatoes'],
 ['Organic Strawberries', 'Organic Baby Spinach'],
 ['Seedless Red Grapes']]

#  association items of all users (remove asiles : fresh fruits(24),fresh vegetables(83),packaged vegetables fruits(122))

### find the top 3 aisles [fresh fruits,fresh vegetables,packaged vegetables fruits]

In [132]:
aisle_ids = []
aisle_arr = aisle[aisle['aisle'].isin(['fresh fruits','fresh vegetables','packaged vegetables fruits'])].values
for item in aisle_arr:
    aisle_ids.append(item[0])

product_ids = []
product_arr = products[products['aisle_id'].isin(aisle_ids)].values
for item in product_arr:
    product_ids.append(item[0])
product_ids
aisle_ids

[24, 83, 123]

In [133]:
user_product_id = user_product_id[~user_product_id['product_id'].isin(product_ids)]

In [135]:
df = Apriori(user_product_id)
t1 = time.time()
frequent_itemsets = apriori(df, min_support=0.08, use_colnames=True)
t2 = time.time()
print('time t2 - t1',t2-t1)

time t2 - t1 31.25018072128296


In [136]:
association_rule = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.08)
# association_rule

In [137]:
transform_associationRule_to_product(association_rule)

[]

In [138]:
s = frequent_itemsets.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(s)

[['Organic Cilantro'],
 ['Extra Virgin Olive Oil'],
 ['Organic Italian Parsley Bunch'],
 ['Blueberries'],
 ['Organic Whole Milk'],
 ['Bunched Cilantro'],
 ['Original Hummus'],
 ['Raspberries'],
 ['Grated Parmesan'],
 ['Sparkling Water Grapefruit'],
 ['Boneless Skinless Chicken Breasts'],
 ['Organic Half & Half'],
 ['Half & Half'],
 ['100% Whole Wheat Bread']]

# after removing frequent asiles and apply association rule on each user classification 

In [15]:
user_classification = pd.read_csv('../customer_classificationV2.csv')
user_classification = user_classification[['user_id','label']]
user0 = user_classification[user_classification['label']==0]
user1 = user_classification[user_classification['label']==1]
user2 = user_classification[user_classification['label']==2]
user3 = user_classification[user_classification['label']==3]
user4 = user_classification[user_classification['label']==4]

In [116]:
def apply_apriori_fun(dataset,minSup,minCon):
    df = Apriori(dataset)
    t1 = time.time()
    frequent_itemsets0 = apriori(df, min_support=minSup, use_colnames=True)
    t2 = time.time()
    print('time t2 - t1',t2-t1)

    association_rule0 = association_rules(frequent_itemsets0, metric="confidence", min_threshold=minCon)
    # association_rule0
    
    s = frequent_itemsets0.sort_values('support',ascending=False)[:20]
    print('top frequent items')
    frequent_items = transform_frequentItem_to_product(s)
    for item in frequent_items:
        print(item)
    
    
    return transform_associationRule_to_product(association_rule0)

#  association items of clustering 0

In [117]:
user_product_part0 = user_product_id[user_product_id['user_id'].isin(user0['user_id']) ]

### top rules and top items of train data (70%)

In [118]:
apply_apriori_fun(user_product_part0[:int(len(user_product_part0)*0.7)],0.08,0.1)

time t2 - t1 2.656677722930908
top frequent items
['Organic Cilantro']
['Organic Italian Parsley Bunch']
['Extra Virgin Olive Oil']
['Organic Whole Milk']
['Blueberries']
['Bunched Cilantro']
['Grated Parmesan']
['Original Hummus']
['Unsalted Butter']
['Half & Half']
['Organic Black Beans']
['Organic Basil']
['Boneless Skinless Chicken Breasts']
['Feta Cheese Crumbles']
['Organic Garbanzo Beans']
['Organic Half & Half']
['Organic Thyme']
['Organic Cilantro', 'Organic Italian Parsley Bunch']
['Organic Coconut Milk']
['Organic Sour Cream']


[(['Organic Cilantro'], '----->', ['Organic Whole Milk']),
 (['Organic Whole Milk'], '----->', ['Organic Cilantro']),
 (['Organic Whole Milk'], '----->', ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], '----->', ['Organic Whole Milk']),
 (['Bunched Cilantro'], '----->', ['Organic Cilantro']),
 (['Organic Cilantro'], '----->', ['Bunched Cilantro']),
 (['Bunched Cilantro'], '----->', ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], '----->', ['Bunched Cilantro']),
 (['Extra Virgin Olive Oil'], '----->', ['Organic Cilantro']),
 (['Organic Cilantro'], '----->', ['Extra Virgin Olive Oil']),
 (['Extra Virgin Olive Oil'], '----->', ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], '----->', ['Extra Virgin Olive Oil']),
 (['Grated Parmesan'], '----->', ['Organic Cilantro']),
 (['Organic Cilantro'], '----->', ['Grated Parmesan']),
 (['Organic Cilantro'], '----->', ['Organic Italian Parsley Bunch']),
 (['Organic Italian Pars

### top rules and top items of test data (30%)

In [119]:
apply_apriori_fun(user_product_part0[-int(len(user_product_part0)*0.3):],0.08,0.1)

time t2 - t1 0.8430626392364502
top frequent items
['Extra Virgin Olive Oil']
['Organic Cilantro']
['Organic Italian Parsley Bunch']
['Organic Whole Milk']
['Blueberries']
['Bunched Cilantro']
['Grated Parmesan']
['Original Hummus']
['Unsalted Butter']
['Organic Black Beans']
['Half & Half']
['Organic Half & Half']
['Boneless Skinless Chicken Breasts']
['Organic Basil']
['Feta Cheese Crumbles']
['Organic Coconut Milk']
['Organic Garbanzo Beans']
['Organic Cilantro', 'Organic Italian Parsley Bunch']
['Organic Whole String Cheese']
['Raspberries']


[(['Organic Whole Milk'], '----->', ['Organic Cilantro']),
 (['Organic Cilantro'], '----->', ['Organic Whole Milk']),
 (['Organic Whole Milk'], '----->', ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], '----->', ['Organic Whole Milk']),
 (['Bunched Cilantro'], '----->', ['Organic Cilantro']),
 (['Organic Cilantro'], '----->', ['Bunched Cilantro']),
 (['Bunched Cilantro'], '----->', ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], '----->', ['Bunched Cilantro']),
 (['Extra Virgin Olive Oil'], '----->', ['Organic Cilantro']),
 (['Organic Cilantro'], '----->', ['Extra Virgin Olive Oil']),
 (['Extra Virgin Olive Oil'], '----->', ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], '----->', ['Extra Virgin Olive Oil']),
 (['Organic Cilantro'], '----->', ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], '----->', ['Organic Cilantro']),
 (['Grated Parmesan'], '----->', ['Organic Italian Parsley Bunch']

#  association items of clustering 1

In [120]:
user_product_part1 = user_product_id[user_product_id['user_id'].isin(user1['user_id']) ]

### top rules and top items of train data (70%)

In [121]:
apply_apriori_fun(user_product_part1[:int(len(user_product_part1)*0.7)],0.18,0.5)

time t2 - t1 0.06337094306945801
top frequent items
['Organic Whole Milk']
['Baby Food Stage 2 Blueberry Pear & Purple Carrot']
['Spinach Peas & Pear Stage 2 Baby Food']
['Organic Whole String Cheese']
['Grated Parmesan']
['Organic Italian Parsley Bunch']
['Extra Virgin Olive Oil']
['Broccoli & Apple Stage 2 Baby Food']
['Original Hummus']
['Organic Cilantro']
['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food']
['Organic Mixed Berry Yogurt & Fruit Snack']
['Blueberries']
['Organic Black Beans']
['Organic Pears, Peas and Broccoli Puree Stage 1']
['Happy Baby Spinach, Mango, and Pear Baby Food']
['Peach,  Apricot & Banana Stage 2 Baby Food']
['Organic Strawberry Yogurt & Fruit Snack']
['Organic Stage 4 Green Bean Pear & Pea Baby Food']
['Feta Cheese Crumbles']


[(['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Spinach Peas & Pear Stage 2 Baby Food']),
 (['Spinach Peas & Pear Stage 2 Baby Food'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),
 (['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food'],
  '----->',
  ['Organic Pears, Peas and Broccoli Puree Stage 1']),
 (['Organic Pears, Peas and Broccoli Puree Stage 1'],
  '----->',
  ['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food']),
 (['Organic Whole String Cheese'], '----->', ['Organic Whole Milk']),
 (['Grated Parmesan'], '----->', ['Organic Whole Milk']),
 (['Organic Italian Parsley Bunch'], '----->', ['Organic Whole Milk']),
 (['Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
  '----->',
  ['Spin

### top rules and top items of test data (30%)

In [122]:
apply_apriori_fun(user_product_part1[-int(len(user_product_part1)*0.3):],0.18,0.5)

time t2 - t1 0.03435707092285156
top frequent items
['Organic Whole Milk']
['Baby Food Stage 2 Blueberry Pear & Purple Carrot']
['Spinach Peas & Pear Stage 2 Baby Food']
['Broccoli & Apple Stage 2 Baby Food']
['Grated Parmesan']
['Organic Whole String Cheese']
['Extra Virgin Olive Oil']
['Organic Italian Parsley Bunch']
['Organic Pears, Peas and Broccoli Puree Stage 1']
['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food']
['Peach,  Apricot & Banana Stage 2 Baby Food']
['Happy Baby Spinach, Mango, and Pear Baby Food']
['Organic Cilantro']
['Sweeet Potato, Apple, Carrot & Cinnamon Organic Superfoods']
['Blueberries']
['Baby Food Stage 2 Pumpkin Banana']
['Baby Food Stage 2 Blueberry Pear & Purple Carrot', 'Spinach Peas & Pear Stage 2 Baby Food']
['Baby Food Stage 2 Blueberry Pear & Purple Carrot', 'Broccoli & Apple Stage 2 Baby Food']
['Organic Stage 4 Spinach Mango & Pear Baby Food']
['Organic Mixed Berry Yogurt & Fruit Snack']


[(['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Spinach Peas & Pear Stage 2 Baby Food']),
 (['Spinach Peas & Pear Stage 2 Baby Food'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),
 (['Peach,  Apricot & Banana Stage 2 Baby Food'],
  '----->',
  ['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),
 (['Organic 4 Months Butternut Squash Carrots Apples + Prunes Baby Food'],
  '----->',
  ['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food']),
 (['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food'],
  '----->',
  ['Organic 4 Months Butternut Squash Carrots Apples + Prunes Baby Food']),
 (['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food'],
  '----->',
  ['Organic Pears, 

#  association items of clustering 2

In [123]:
user_product_part2 = user_product_id[user_product_id['user_id'].isin(user1['user_id']) ]

### top rules and top items of train data (70%)

In [124]:
apply_apriori_fun(user_product_part2[:int(len(user_product_part2)*0.7)],0.15,0.15)

time t2 - t1 0.0791170597076416
top frequent items
['Organic Whole Milk']
['Baby Food Stage 2 Blueberry Pear & Purple Carrot']
['Spinach Peas & Pear Stage 2 Baby Food']
['Organic Whole String Cheese']
['Grated Parmesan']
['Organic Italian Parsley Bunch']
['Extra Virgin Olive Oil']
['Broccoli & Apple Stage 2 Baby Food']
['Original Hummus']
['Organic Cilantro']
['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food']
['Organic Mixed Berry Yogurt & Fruit Snack']
['Blueberries']
['Organic Pears, Peas and Broccoli Puree Stage 1']
['Organic Black Beans']
['Happy Baby Spinach, Mango, and Pear Baby Food']
['Peach,  Apricot & Banana Stage 2 Baby Food']
['Organic Strawberry Yogurt & Fruit Snack']
['Organic Stage 4 Green Bean Pear & Pea Baby Food']
['Feta Cheese Crumbles']


[(['Broccoli & Apple Stage 2 Baby Food'], '----->', ['Organic Whole Milk']),
 (['Organic Whole Milk'], '----->', ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Spinach Peas & Pear Stage 2 Baby Food']),
 (['Spinach Peas & Pear Stage 2 Baby Food'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),
 (['Peach,  Apricot & Banana Stage 2 Baby Food'],
  '----->',
  ['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),
 (['Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
  '----->',
  ['Peach,  Apricot & Banana Stage 2 Baby Food']),
 (['Organic 4 Months Butternut Squash Carrots Apples + Prunes Baby Food'],
  '----->',
  ['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food']),
 (['Stage 1 Ap

### top rules and top items of test data (30%)

In [125]:
apply_apriori_fun(user_product_part2[-int(len(user_product_part2)*0.3):],0.15,0.15)

time t2 - t1 0.039346933364868164
top frequent items
['Organic Whole Milk']
['Baby Food Stage 2 Blueberry Pear & Purple Carrot']
['Spinach Peas & Pear Stage 2 Baby Food']
['Broccoli & Apple Stage 2 Baby Food']
['Grated Parmesan']
['Organic Whole String Cheese']
['Extra Virgin Olive Oil']
['Organic Italian Parsley Bunch']
['Organic Pears, Peas and Broccoli Puree Stage 1']
['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food']
['Peach,  Apricot & Banana Stage 2 Baby Food']
['Happy Baby Spinach, Mango, and Pear Baby Food']
['Organic Cilantro']
['Sweeet Potato, Apple, Carrot & Cinnamon Organic Superfoods']
['Blueberries']
['Baby Food Stage 2 Pumpkin Banana']
['Baby Food Stage 2 Blueberry Pear & Purple Carrot', 'Spinach Peas & Pear Stage 2 Baby Food']
['Organic Stage 4 Spinach Mango & Pear Baby Food']
['Baby Food Stage 2 Blueberry Pear & Purple Carrot', 'Broccoli & Apple Stage 2 Baby Food']
['Organic Mixed Berry Yogurt & Fruit Snack']


[(['Peach,  Apricot & Banana Stage 2 Baby Food'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Peach,  Apricot & Banana Stage 2 Baby Food']),
 (['Baby Food Stage 2 Pumpkin Banana'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Baby Food Stage 2 Pumpkin Banana']),
 (['Broccoli & Apple Stage 2 Baby Food'], '----->', ['Organic Whole Milk']),
 (['Organic Whole Milk'], '----->', ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Spinach Peas & Pear Stage 2 Baby Food']),
 (['Spinach Peas & Pear Stage 2 Baby Food'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),
 (['Apple and Carrot Sta

#  association items of clustering 3

In [126]:
user_product_part3 = user_product_id[user_product_id['user_id'].isin(user3['user_id']) ]

### top rules and top items of train data (70%)

In [127]:
apply_apriori_fun(user_product_part3[:int(len(user_product_part3)*0.7)],0.25,0.25)

time t2 - t1 0.013834953308105469
top frequent items
['Hass Avocados']
['Clementines']
['Raspberries']
['Packaged Grape Tomatoes']
['Seedless Cucumbers']
['Clementines', 'Hass Avocados']
['Raspberries', 'Hass Avocados']
['Clementines', 'Raspberries']
['Rainbow Bell Peppers']
['Baby Cucumbers']
['Packaged Grape Tomatoes', 'Hass Avocados']
['Seedless Cucumbers', 'Hass Avocados']
['Packaged Grape Tomatoes', 'Clementines']
['Green Seedless Grapes']
['Organic Tortilla Chips']
['Rainbow Bell Peppers', 'Hass Avocados']
['Packaged Grape Tomatoes', 'Raspberries']
['Blueberries']
['Clementines', 'Raspberries', 'Hass Avocados']
['Raspberries', 'Seedless Cucumbers']


[(['Packaged Grape Tomatoes'], '----->', ['Clementines']),
 (['Clementines'], '----->', ['Packaged Grape Tomatoes']),
 (['Clementines'], '----->', ['Baby Cucumbers']),
 (['Baby Cucumbers'], '----->', ['Clementines']),
 (['Clementines'], '----->', ['Hass Avocados']),
 (['Hass Avocados'], '----->', ['Clementines']),
 (['Clementines'], '----->', ['Seedless Cucumbers']),
 (['Seedless Cucumbers'], '----->', ['Clementines']),
 (['Clementines'], '----->', ['Raspberries']),
 (['Raspberries'], '----->', ['Clementines']),
 (['Packaged Grape Tomatoes'], '----->', ['Hass Avocados']),
 (['Hass Avocados'], '----->', ['Packaged Grape Tomatoes']),
 (['Packaged Grape Tomatoes'], '----->', ['Rainbow Bell Peppers']),
 (['Rainbow Bell Peppers'], '----->', ['Packaged Grape Tomatoes']),
 (['Packaged Grape Tomatoes'], '----->', ['Seedless Cucumbers']),
 (['Seedless Cucumbers'], '----->', ['Packaged Grape Tomatoes']),
 (['Packaged Grape Tomatoes'], '----->', ['Raspberries']),
 (['Raspberries'], '----->', ['Pa

### top rules and top items of test data (30%)

In [128]:
apply_apriori_fun(user_product_part3[-int(len(user_product_part3)*0.3):],0.25,0.25)

time t2 - t1 0.009364128112792969
top frequent items
['Hass Avocados']
['Clementines']
['Raspberries']
['Packaged Grape Tomatoes']
['Clementines', 'Hass Avocados']
['Baby Cucumbers']
['Seedless Cucumbers']
['Raspberries', 'Hass Avocados']
['Raspberries', 'Clementines']
['Rainbow Bell Peppers']
['Packaged Grape Tomatoes', 'Hass Avocados']
['Packaged Grape Tomatoes', 'Clementines']
['Blueberries']
['Seedless Cucumbers', 'Hass Avocados']
['Baby Cucumbers', 'Hass Avocados']
['Packaged Grape Tomatoes', 'Raspberries']
['Clementines', 'Baby Cucumbers']
['Rainbow Bell Peppers', 'Hass Avocados']
['Broccoli Florettes']
['Clementines', 'Raspberries', 'Hass Avocados']


[(['Packaged Grape Tomatoes'], '----->', ['Clementines']),
 (['Clementines'], '----->', ['Packaged Grape Tomatoes']),
 (['Clementines'], '----->', ['Baby Cucumbers']),
 (['Baby Cucumbers'], '----->', ['Clementines']),
 (['Clementines'], '----->', ['Hass Avocados']),
 (['Hass Avocados'], '----->', ['Clementines']),
 (['Clementines'], '----->', ['Rainbow Bell Peppers']),
 (['Rainbow Bell Peppers'], '----->', ['Clementines']),
 (['Clementines'], '----->', ['Seedless Cucumbers']),
 (['Seedless Cucumbers'], '----->', ['Clementines']),
 (['Raspberries'], '----->', ['Clementines']),
 (['Clementines'], '----->', ['Raspberries']),
 (['Packaged Grape Tomatoes'], '----->', ['Baby Cucumbers']),
 (['Baby Cucumbers'], '----->', ['Packaged Grape Tomatoes']),
 (['Packaged Grape Tomatoes'], '----->', ['Hass Avocados']),
 (['Hass Avocados'], '----->', ['Packaged Grape Tomatoes']),
 (['Packaged Grape Tomatoes'], '----->', ['Rainbow Bell Peppers']),
 (['Rainbow Bell Peppers'], '----->', ['Packaged Grape T

#  association items of clustering 4

In [129]:
user_product_part4 = user_product_id[user_product_id['user_id'].isin(user4['user_id']) ]

In [130]:
apply_apriori_fun(user_product_part4[:int(len(user_product_part4)*0.7)],0.12,0.12)

time t2 - t1 0.1604328155517578
top frequent items
['Sparkling Water Grapefruit']
['Lime Sparkling Water']
['Extra Virgin Olive Oil']
['Sparkling Lemon Water']
['Sparkling Water Grapefruit', 'Lime Sparkling Water']
['Sparkling Natural Mineral Water']
['Organic Italian Parsley Bunch']
['Organic Cilantro']
['Organic Half & Half']
['Spring Water']
['Half & Half']
['Blueberries']
['Original Hummus']
['Peach Pear Flavored Sparkling Water']
['Sparkling Water Grapefruit', 'Sparkling Lemon Water']
['Sparkling Lemon Water', 'Lime Sparkling Water']
['Organic Whole Milk']
['Sparkling Water Berry']
['100% Recycled Paper Towels']
['Grated Parmesan']


[(['Lime Sparkling Water'], '----->', ['Sparkling Water Berry']),
 (['Sparkling Water Berry'], '----->', ['Lime Sparkling Water']),
 (['Sparkling Water Grapefruit'], '----->', ['Sparkling Water Berry']),
 (['Sparkling Water Berry'], '----->', ['Sparkling Water Grapefruit']),
 (['Sparkling Lemon Water'], '----->', ['Lime Sparkling Water']),
 (['Lime Sparkling Water'], '----->', ['Sparkling Lemon Water']),
 (['Sparkling Water Grapefruit'], '----->', ['Sparkling Lemon Water']),
 (['Sparkling Lemon Water'], '----->', ['Sparkling Water Grapefruit']),
 (['Peach Pear Flavored Sparkling Water'], '----->', ['Lime Sparkling Water']),
 (['Lime Sparkling Water'], '----->', ['Peach Pear Flavored Sparkling Water']),
 (['Sparkling Water Grapefruit'],
  '----->',
  ['Peach Pear Flavored Sparkling Water']),
 (['Peach Pear Flavored Sparkling Water'],
  '----->',
  ['Sparkling Water Grapefruit']),
 (['Sparkling Water Grapefruit'], '----->', ['Lime Sparkling Water']),
 (['Lime Sparkling Water'], '----->',

In [131]:
apply_apriori_fun(user_product_part4[-int(len(user_product_part4)*0.3):],0.12,0.12)

time t2 - t1 0.07832908630371094
top frequent items
['Sparkling Water Grapefruit']
['Lime Sparkling Water']
['Extra Virgin Olive Oil']
['Sparkling Lemon Water']
['Organic Half & Half']
['Sparkling Water Grapefruit', 'Lime Sparkling Water']
['Half & Half']
['Original Hummus']
['Peach Pear Flavored Sparkling Water']
['Organic Cilantro']
['Organic Italian Parsley Bunch']
['Blueberries']
['Sparkling Natural Mineral Water']
['Sparkling Water Grapefruit', 'Sparkling Lemon Water']
['Sparkling Water Berry']
['Sparkling Water Grapefruit', 'Peach Pear Flavored Sparkling Water']
['Pure Sparkling Water']
['Sparkling Lemon Water', 'Lime Sparkling Water']
['Organic Whole Milk']
['Spring Water']


[(['Sparkling Water Grapefruit'], '----->', ['Pure Sparkling Water']),
 (['Pure Sparkling Water'], '----->', ['Sparkling Water Grapefruit']),
 (['Peach Pear Flavored Sparkling Water'],
  '----->',
  ['Sparkling Water Berry']),
 (['Sparkling Water Berry'],
  '----->',
  ['Peach Pear Flavored Sparkling Water']),
 (['Lime Sparkling Water'], '----->', ['Sparkling Water Berry']),
 (['Sparkling Water Berry'], '----->', ['Lime Sparkling Water']),
 (['Sparkling Water Grapefruit'], '----->', ['Sparkling Water Berry']),
 (['Sparkling Water Berry'], '----->', ['Sparkling Water Grapefruit']),
 (['Sparkling Lemon Water'], '----->', ['Lime Sparkling Water']),
 (['Lime Sparkling Water'], '----->', ['Sparkling Lemon Water']),
 (['Sparkling Water Grapefruit'], '----->', ['Sparkling Lemon Water']),
 (['Sparkling Lemon Water'], '----->', ['Sparkling Water Grapefruit']),
 (['Peach Pear Flavored Sparkling Water'], '----->', ['Lime Sparkling Water']),
 (['Lime Sparkling Water'], '----->', ['Peach Pear Flavo